<a href="https://colab.research.google.com/github/bukolastella/Data-Science/blob/master/Deep%20Learning/from%20tensorflow/patch_camelyon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow import keras
import tensorflow as tf
import os,datetime
import tensorflow_datasets as tfds

In [ ]:
df, info = tfds.load('patch_camelyon', with_info = True, as_supervised = True)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is be

Shuffling and writing examples to /root/tensorflow_datasets/patch_camelyon/2.0.0.incomplete61M2AZ/patch_camelyon-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/patch_camelyon/2.0.0.incomplete61M2AZ/patch_camelyon-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/patch_camelyon/2.0.0.incomplete61M2AZ/patch_camelyon-validation.tfrecord


Dataset patch_camelyon downloaded and prepared to /root/tensorflow_datasets/patch_camelyon/2.0.0. Subsequent calls will reuse this data.


In [ ]:
df

{'test': <DatasetV1Adapter shapes: ((96, 96, 3), ()), types: (tf.uint8, tf.int64)>,
 'train': <DatasetV1Adapter shapes: ((96, 96, 3), ()), types: (tf.uint8, tf.int64)>,
 'validation': <DatasetV1Adapter shapes: ((96, 96, 3), ()), types: (tf.uint8, tf.int64)>}

In [ ]:
train_data = df['train']
valid_data = df['validation']
test_data = df['test']

In [ ]:
#train_data = train_data.prefetch(buffer_size=32)
#valid_data = valid_data.prefetch(buffer_size=32)


In [ ]:
def preprocess(image, labels):
  image = tf.cast(image, tf.float32)
  image /= 255.
  return image, labels

In [ ]:
train_data = train_data.map(preprocess)
valid_data = valid_data.map(preprocess)
test_data = test_data.map(preprocess)

In [ ]:
train_data = train_data.shuffle(1000)

In [ ]:
train_data = train_data.batch(128).prefetch(1)
valid_data = valid_data.batch(128).prefetch(1)
test_data = test_data.batch(128).prefetch(1)

In [ ]:
train_images, train_labels = next(iter(train_data))
valid_images, valid_labels = next(iter(valid_data))
test_images, test_labels  = next(iter(test_data))

In [ ]:
'''import numpy as np
train_labels = np.asarray(train_labels).astype('float32').reshape((-1,1))
test_labels = np.asarray(test_labels).astype('float32').reshape((-1,1))
valid_labels = np.asarray(valid_labels).astype('float32').reshape((-1,1))'''

"import numpy as np\ntrain_labels = np.asarray(train_labels).astype('float32').reshape((-1,1))\ntest_labels = np.asarray(test_labels).astype('float32').reshape((-1,1))\nvalid_labels = np.asarray(valid_labels).astype('float32').reshape((-1,1))"

In [ ]:
valid_labels.shape

TensorShape([128])

In [ ]:
train_images.shape

TensorShape([128, 96, 96, 3])

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip= True)

it_gen = datagen.flow(train_images,  train_labels, batch_size=32)


Using TensorFlow backend.


**MODEL**

In [ ]:
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers

model = Sequential([
                    Conv2D(256, 3,padding='same', kernel_initializer='he_uniform', activation='relu', input_shape = [96, 96, 3]),
                    MaxPooling2D(2),
                    Conv2D(256, 3,padding='same', kernel_initializer='he_uniform',activation='relu',),
                    MaxPooling2D(2),
                    Conv2D(512, 3,padding='same',kernel_initializer='he_uniform',activation='relu',),
                    MaxPooling2D(2),
                    Conv2D(512, 3,padding='same',kernel_initializer='he_uniform',activation='relu',),
                    MaxPooling2D(2),
                    Conv2D(1024, 3,padding='same', kernel_initializer='he_uniform',activation='relu',),
                    MaxPooling2D(2),
                    Conv2D(1024, 3,padding='same', kernel_initializer='he_uniform',activation='relu',),
                    MaxPooling2D(2),
                    Flatten(),
                    Dense(1028,kernel_initializer='he_uniform',activation = 'relu'),
                    Dense(512,kernel_initializer='he_uniform',activation = 'relu'),
                    Dense(128, kernel_initializer='he_uniform',activation = 'relu'),
                    Dense(1, activation = 'sigmoid'),
                    ])
 

In [ ]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 96, 96, 256)       7168      
_________________________________________________________________
max_pooling2d_30 (MaxPooling (None, 48, 48, 256)       0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 48, 48, 256)       590080    
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 24, 24, 256)       0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 24, 24, 512)       1180160   
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 12, 12, 512)       0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 12, 12, 512)      

In [ ]:
model.compile(optimizer= optimizers.Adam(1e-4), loss='binary_crossentropy', metrics=['acc'])
steps = int(train_images.shape[0] / 32)
early_stopping_cb = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
history = model.fit( train_images, train_labels, epochs = 100, callbacks=[early_stopping_cb], validation_data = (valid_images, valid_labels), verbose=2)


Epoch 1/100
4/4 - 1s - loss: 1.8183 - acc: 0.5234 - val_loss: 1.8839 - val_acc: 0.5859
Epoch 2/100
4/4 - 0s - loss: 1.3978 - acc: 0.4766 - val_loss: 0.7772 - val_acc: 0.5859
Epoch 3/100
4/4 - 0s - loss: 0.8001 - acc: 0.5703 - val_loss: 0.9322 - val_acc: 0.4141
Epoch 4/100
4/4 - 0s - loss: 0.8204 - acc: 0.4766 - val_loss: 0.6860 - val_acc: 0.5859
Epoch 5/100
4/4 - 0s - loss: 0.7237 - acc: 0.4922 - val_loss: 0.6811 - val_acc: 0.5859
Epoch 6/100
4/4 - 0s - loss: 0.6863 - acc: 0.6250 - val_loss: 0.6298 - val_acc: 0.5859
Epoch 7/100
4/4 - 0s - loss: 0.6624 - acc: 0.6016 - val_loss: 0.6694 - val_acc: 0.6172
Epoch 8/100
4/4 - 0s - loss: 0.6208 - acc: 0.6797 - val_loss: 0.6058 - val_acc: 0.6484
Epoch 9/100
4/4 - 0s - loss: 0.6142 - acc: 0.6172 - val_loss: 0.5925 - val_acc: 0.6719
Epoch 10/100
4/4 - 0s - loss: 0.5797 - acc: 0.6797 - val_loss: 0.5595 - val_acc: 0.7344
Epoch 11/100
4/4 - 0s - loss: 0.5660 - acc: 0.7109 - val_loss: 0.5446 - val_acc: 0.7188
Epoch 12/100
4/4 - 0s - loss: 0.5412 - ac

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

NameError: ignored

**TRANSFER LEARNING**

In [ ]:
model.evaluate(test_images, test_labels)